In [1]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.metrics import mean_squared_error,accuracy_score,mean_absolute_error
from sklearn.neural_network import MLPRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import ElasticNet, Lasso,Ridge, LinearRegression,HuberRegressor,SGDRegressor,TweedieRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR,SVR
from sklearn.metrics import mean_squared_error,explained_variance_score,r2_score
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor, AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor,VotingRegressor
import xgboost as xgb
import os
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_regression
from sklearn.decomposition import FastICA

In [2]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM, GRU,Bidirectional,TimeDistributed, ConvLSTM2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.metrics import RootMeanSquaredError,mean_absolute_percentage_error
from keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout
from keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [21]:
# split a univariate time series into patterns
def get_Patterns(TSeries, n_inputs,h):
    X,y,z = pd.DataFrame(np.zeros((len(TSeries)-n_inputs-h+1,n_inputs))), pd.DataFrame(), pd.DataFrame()
    for i in range(len(TSeries)):
        # find the end of this pattern
        end_ix = i + n_inputs + h - 1
        # check if we are beyond the time series
        if end_ix > len(TSeries)-1:
            break
        # gather input and output parts of the pattern
        for j in range(n_inputs):
            X.loc[i,j]=TSeries.iloc[i+j,0]
        i=i+n_inputs
        y=y.append(TSeries.iloc[end_ix], ignore_index = True)
        sinX=pd.DataFrame(np.sin(X))
        cosX=pd.DataFrame(np.cos(X))
        squareX=pd.DataFrame(np.power(X,2))
        X1=np.hstack((X,sinX,cosX,squareX))
    return np.array(X1),np.array(y)

In [4]:
# originalData should be a Column Vectored DataFrame
def minmaxNorm(originalData, lenTrainValidation):
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    normalizedData=np.zeros(lenOriginal)   
    normalizedData = []
    for i in range (lenOriginal):
        normalizedData.append((originalData.iloc[i]-min2norm)/(max2norm-min2norm))    
    return pd.DataFrame(normalizedData)

In [5]:
# originalData and forecastedData should be Column Vectored DataFrames
def minmaxDeNorm( originalData, forecastedData, lenTrainValidation):
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    denormalizedData=[]    
    for i in range (lenOriginal):
        denormalizedData.append((forecastedData.iloc[i]*(max2norm-min2norm))+min2norm)  
    return pd.DataFrame(denormalizedData)

In [6]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findRMSE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainRMSE=0;
    for i in range (lenTrainValidation):
        trainRMSE=trainRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2) 
    trainRMSE=np.sqrt(trainRMSE/lenTrainValidation)

    testRMSE=0;
    for i in range (lenTrainValidation,l,1):
        testRMSE=testRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2)
    testRMSE=np.sqrt(testRMSE/lenTest)
    return trainRMSE, testRMSE 

In [7]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findSMAPE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainSMAPE=0;
    for i in range (lenTrainValidation):
        trainSMAPE=trainSMAPE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/((np.abs(forecasted_value.iloc[i,0])+np.abs(Timeseries_Data.iloc[i,0]))/2))
    
    trainSMAPE=(trainSMAPE/(lenTrainValidation))*100;

    testSMAPE=0;
    for i in range (lenTrainValidation,l,1):
        testSMAPE=testSMAPE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/((np.abs(forecasted_value.iloc[i,0])+np.abs(Timeseries_Data.iloc[i,0]))/2))
    
    testSMAPE=(testSMAPE/lenTest)*100;
    return trainSMAPE, testSMAPE

In [8]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findMAE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainMAE=0;
    for i in range (lenTrainValidation):
        trainMAE=trainMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]) 
    trainMAE=(trainMAE/(lenTrainValidation));

    testMAE=0;
    for i in range (lenTrainValidation,l,1):
        testMAE=testMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])
    testMAE=(testMAE/lenTest);
    return trainMAE, testMAE

In [9]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findMASE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainMASE=0;
    T1=Timeseries_Data.iloc[0:lenTrainValidation,0]
    T2=T1.diff()
    T2=T2.drop(0)
    T2=np.abs(T2)
    M=np.mean(T2)
    for i in range (lenTrainValidation):
        trainMASE=trainMASE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/M) 

    trainMASE=(trainMASE/(lenTrainValidation))

    testMASE=0
    T1=Timeseries_Data.iloc[lenTrainValidation:l,0]
    T2=T1.diff()
    T2=T2.drop(lenTrainValidation)
    T2=np.abs(T2)
    M=np.mean(T2)
    for i in range (lenTrainValidation,l,1):
        testMASE=testMASE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/M)
    testMASE=(testMASE/lenTest)
    return trainMASE,testMASE 

In [10]:
def get_Activation_Function(no):
    if no==0:
        return "relu"
    elif no==1:
        return "sigmoid"
    elif no==2:
        return "tanh"
    elif no==3:
        return "selu"
    elif no==4:
        return "exponential"
    else:
        return "elu"

In [11]:
def get_kernel_initializer(no):
    if no==0:
        return "glorot_uniform"
    elif no==1:
        return "glorot_normal"
    elif no==2:
        return "he_uniform"
    else:
        return "he_normal"

In [12]:
def Find_Features_LSTM(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    lenTrain=NOP-lenValid-lenTest
    LagLength=x.shape[1]
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model1 = Sequential()
    model1.add(LSTM(256, activation='tanh', input_shape=(LagLength,n_features),kernel_initializer="he_normal",name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal",name="layer2"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.0001),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=100, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    feature_extractor = Model(inputs=model1.inputs, outputs=model1.get_layer(name="layer2").output)
    features = pd.DataFrame(feature_extractor.predict(x))
    return features

In [13]:
def Find_Features_CNN(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    LagLength=x.shape[1]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    x = x.reshape((x.shape[0], x.shape[1], n_features))
    model1 = Sequential()
    model1.add(Conv1D(filters=256, kernel_size=2, activation='tanh', input_shape=(LagLength,n_features),kernel_initializer="he_normal",name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal",name="layer2"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.0001),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=100, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    feature_extractor = Model(inputs=model1.inputs, outputs=model1.get_layer(name="layer2").output)
    features = pd.DataFrame(feature_extractor.predict(x).reshape(x.shape[0],16))
    return features

In [14]:
def Find_Features_ConvLSTM(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    n_seq=1
    LagLength=x.shape[1]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    xTrain=xTrain.reshape((xTrain.shape[0], n_seq, 1, LagLength, n_features))
    xValid=xValid.reshape((xValid.shape[0], n_seq, 1, LagLength, n_features))
    xTest=xTest.reshape((xTest.shape[0], n_seq, 1, LagLength, n_features))
    x=x.reshape((x.shape[0], n_seq, 1, LagLength, n_features))
    model1 = Sequential()
    model1.add(ConvLSTM2D(filters=256, kernel_size=(1,2), activation='tanh',kernel_initializer="he_normal" , input_shape=(n_seq,1, LagLength, n_features),name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal",name="layer2"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.0001),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=100, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    feature_extractor = Model(inputs=model1.inputs, outputs=model1.get_layer(name="layer2").output)
    features = pd.DataFrame(feature_extractor.predict(x).reshape(x.shape[0],16))
    return features

In [15]:
def Find_Features_BiLSTM(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    LagLength=x.shape[1]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model1 = Sequential()
    model1.add(Bidirectional(LSTM(256, activation='tanh',kernel_initializer="he_normal"),input_shape=(LagLength,n_features),name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal",name="layer2"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.0001),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=100, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    feature_extractor = Model(inputs=model1.inputs, outputs=model1.get_layer(name="layer2").output)
    features = pd.DataFrame(feature_extractor.predict(x))
    return features

In [16]:
def Find_Features_GRU(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    lenTrain=NOP-lenValid-lenTest
    LagLength=x.shape[1]
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model1 = Sequential()
    model1.add(GRU(256, activation='tanh', input_shape=(LagLength,n_features),kernel_initializer="he_normal",name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=100, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    feature_extractor = Model(inputs=model1.inputs, outputs=model1.get_layer(name="layer1").output)
    features = pd.DataFrame(feature_extractor.predict(x))
    return features

In [17]:
def Find_Fitness(x,y,lenValid,lenTest,model):
    NOP=y.shape[0]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:(lenTrain+lenValid),:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:(lenTrain+lenValid),0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model.fit(xTrain, yTrain)
    yhatNorm=model.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [18]:
def findUOD(TimeSeries):
    Dmin=np.min(TimeSeries)
    Dmax=np.max(TimeSeries)
    D1=0.2*(Dmax-Dmin)
    D2=0.2*(Dmax-Dmin)
    U=pd.DataFrame(np.zeros((1,2)))
    U.iloc[0,0]=Dmin-D1
    U.iloc[0,1]=Dmax+D2
    return pd.DataFrame(U)

In [27]:
#Read the Time Series Dataset
Timeseries_Data=pd.read_csv('AQI.csv',header=None)
linear_Data=pd.read_csv('ARIMA.csv',header=None)
Residual_Data=pd.DataFrame(np.zeros((Timeseries_Data.shape[0],1)))
LagLength=24
h=1
NumberOfRepeatitions=20
lt=Timeseries_Data.shape[0]
lenTrain=int(round(lt*0.6))
lenValidation=int(round(lt*0.2))
lenTest=int(lt-lenTrain-lenValidation)
#Compute the Residual Series
for i in range(lt):
    Residual_Data.iloc[i,0]=Timeseries_Data.iloc[i,0]-linear_Data.iloc[i,0]
# NORMALIZE THE DATA
normalizedData=minmaxNorm(Residual_Data,lenTrain+lenValidation);
# Transform the Time Series into Patterns Using Sliding Window
X, y = get_Patterns(normalizedData, LagLength, h)
CNNFeatures=Find_Features_CNN(X,y,lenValidation,lenTest)
ConvLSTMFeatures=Find_Features_ConvLSTM(X,y,lenValidation,lenTest)
BiLSTMFeatures=Find_Features_BiLSTM(X,y,lenValidation,lenTest)
LSTMFeatures=Find_Features_LSTM(X,y,lenValidation,lenTest)
X=np.hstack((CNNFeatures,LSTMFeatures,BiLSTMFeatures,ConvLSTMFeatures))
#X = SelectKBest(score_func=f_regression, k=24).fit_transform(X, y)
pca = PCA(n_components=9)
X = pca.fit_transform(X)

#ICA = FastICA(n_components=6)
#X=ICA.fit_transform(X)

MeanAccuracyT=pd.DataFrame()
StdAccuracyT=pd.DataFrame()
for j in range(0,21,1):
    print('Model:::::::::::::::::::::::::::::::',j)
    if j==0:         
        model=LinearRegression()
        name='1.LinearRegression'
    elif j==1:
        model=Lasso()
        name='2.Lasso'
    elif j==2:
        model=Ridge()
        name='3.Ridge'
    elif j==3:
        model=ElasticNet()
        name='4.ElasticNet'
    elif j==4:
        model=HuberRegressor()
        name='5.HuberRegressor'
    elif j==5:
        model=SGDRegressor()
        name='6.SGDRegressor'
    elif j==6:
        model=TweedieRegressor()
        name='7.TweedieRegressor'
    elif j==7:
        model=AdaBoostRegressor()
        name='8.AdaBoostRegressor'
    elif j==8:
        model=RandomForestRegressor()
        name='9.RandomForestRegressor'
    elif j==9:
        model=GradientBoostingRegressor()
        name='10.GradientBoostingRegressor'
    elif j==10:
        model=LinearSVR()
        name='11.LinearSVR'
    elif j==11:
        model=MLPRegressor()
        name='12.MLP'
    elif j==12:
        model=SVR()
        name='13.SVR'
    elif j==13:
        model=ExtraTreesRegressor()
        name='14.ExtraTreesRegressor'
    elif j==14:
        model=BaggingRegressor()
        name='15.BaggingRegressor'
    elif j==15:
        model=DecisionTreeRegressor()
        name='16.DecisionTreeRegressor'
    elif j==16:
        model=KNeighborsRegressor()
        name='17.KNeighborsRegressor'
    elif j==17:
        model=xgb.XGBRegressor(silent=True)
        name='18.XGB'
    elif j==18:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=xgb.XGBRegressor(silent=True))
        name='19.Lasso_Tweedi_xgb'
    elif j==19:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=LinearSVR())
        name='20.Lasso_Tweedi_LinearSVR'
    elif j==20:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=Lasso())
        name='21.Lasso_Tweedi_Lasso'
    elif j==21:
        name='22.LSTM'
    elif j==22:
        name='23.CNN'
    elif j==23:
        name='24.DMLP' 
    elif j==24:
        name='25.GRU'
    else:
        print('Completed.....................')

    file1='./'+str(name)+"_Accuracy.xlsx"
    file2='./'+str(name)+"_Forecasts.xlsx"
    Forecasts=pd.DataFrame()
    Accuracy=pd.DataFrame()
    for i in range(NumberOfRepeatitions):
        print('No. of repeatitions completed:',i)
        if j<=20:
            ynorm1=Find_Fitness(X,y,lenValidation,lenTest,model)
        elif j==21:
            ynorm1=Find_Fitness_LSTM(X,y,lenValidation,lenTest)
        elif j==22:
            ynorm1=Find_Fitness_CNN(X,y,lenValidation,lenTest)
        elif j==23:
            ynorm1=Find_Fitness_DMLP(X,y,lenValidation,lenTest)
        else:
            ynorm1=Find_Fitness_GRU(X,y,lenValidation,lenTest)
        ynorm=pd.DataFrame(normalizedData.iloc[0:LagLength,0])
        ynorm=ynorm.append(ynorm1,ignore_index = True)
        yhat2=minmaxDeNorm(Residual_Data, ynorm, lenTrain+lenValidation)
        yhat=pd.DataFrame(np.zeros((Timeseries_Data.shape[0],1)))
        for ii in range(lt):
            yhat.iloc[ii,0]=yhat2.iloc[ii,0]+linear_Data.iloc[ii,0]
        Accuracy.loc[i,0],Accuracy.loc[i,1]=findRMSE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,2],Accuracy.loc[i,3]=findSMAPE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,4],Accuracy.loc[i,5]=findMAE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,6],Accuracy.loc[i,7]=findMASE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Forecasts=Forecasts.append(yhat.T,ignore_index = True)
    Accuracy.to_excel(file1,sheet_name='Accuracy',index=False)
    Forecasts.T.to_excel(file2,sheet_name='Forecasts',index=False)
    MeanAccuracy=pd.DataFrame(np.mean(Accuracy))    
    MeanAccuracyT=MeanAccuracyT.append(MeanAccuracy.T, ignore_index = True)
    StdAccuracy=pd.DataFrame(np.std(Accuracy))    
    StdAccuracyT=StdAccuracyT.append(StdAccuracy.T, ignore_index = True)
    del Accuracy
    del Forecasts
    del MeanAccuracy
    del StdAccuracy
MeanAccuracyT.to_excel('All_Model_Mean_Accuracy.xlsx',sheet_name='All_Model_Accuracy',index=False)
del MeanAccuracyT
StdAccuracyT.to_excel('All_Model_Stdev_Accuracy.xlsx',sheet_name='All_Model_Stdev_Accuracy',index=False)
del StdAccuracyT

Epoch 1/100
1806/1806 [==============================] - 10s 5ms/step - loss: 0.0079 - root_mean_squared_error: 0.0890 - val_loss: 1.4327e-04 - val_root_mean_squared_error: 0.0120
Epoch 2/100
1806/1806 [==============================] - 10s 5ms/step - loss: 0.0012 - root_mean_squared_error: 0.0345 - val_loss: 0.0020 - val_root_mean_squared_error: 0.0451
Epoch 3/100
1806/1806 [==============================] - 10s 5ms/step - loss: 0.0013 - root_mean_squared_error: 0.0361 - val_loss: 2.6203e-04 - val_root_mean_squared_error: 0.0162
Epoch 4/100
1806/1806 [==============================] - 10s 5ms/step - loss: 0.0013 - root_mean_squared_error: 0.0359 - val_loss: 2.5056e-04 - val_root_mean_squared_error: 0.0158
Epoch 5/100
1806/1806 [==============================] - 10s 5ms/step - loss: 0.0013 - root_mean_squared_error: 0.0360 - val_loss: 1.2689e-04 - val_root_mean_squared_error: 0.0113
Epoch 6/100
1806/1806 [==============================] - 10s 5ms/step - loss: 0.0012 - root_mean_squared

Epoch 46/100
1806/1806 [==============================] - 10s 5ms/step - loss: 6.6055e-04 - root_mean_squared_error: 0.0257 - val_loss: 6.7112e-05 - val_root_mean_squared_error: 0.0082
Epoch 47/100
1806/1806 [==============================] - 10s 5ms/step - loss: 6.6055e-04 - root_mean_squared_error: 0.0257 - val_loss: 6.6445e-05 - val_root_mean_squared_error: 0.0082
Epoch 48/100
1806/1806 [==============================] - 10s 5ms/step - loss: 6.6042e-04 - root_mean_squared_error: 0.0257 - val_loss: 6.6086e-05 - val_root_mean_squared_error: 0.0081
Epoch 49/100
1806/1806 [==============================] - 10s 5ms/step - loss: 6.6054e-04 - root_mean_squared_error: 0.0257 - val_loss: 6.6087e-05 - val_root_mean_squared_error: 0.0081
Epoch 50/100
1806/1806 [==============================] - 10s 5ms/step - loss: 6.6031e-04 - root_mean_squared_error: 0.0257 - val_loss: 6.8194e-05 - val_root_mean_squared_error: 0.0083
Epoch 51/100
1806/1806 [==============================] - 10s 6ms/step - lo

1806/1806 [==============================] - 10s 5ms/step - loss: 6.6066e-04 - root_mean_squared_error: 0.0257 - val_loss: 6.6313e-05 - val_root_mean_squared_error: 0.0081
Epoch 91/100
1806/1806 [==============================] - 10s 5ms/step - loss: 6.6035e-04 - root_mean_squared_error: 0.0257 - val_loss: 6.6053e-05 - val_root_mean_squared_error: 0.0081
Epoch 92/100
1806/1806 [==============================] - 10s 5ms/step - loss: 6.6051e-04 - root_mean_squared_error: 0.0257 - val_loss: 6.6070e-05 - val_root_mean_squared_error: 0.0081
Epoch 93/100
1806/1806 [==============================] - 10s 5ms/step - loss: 6.6051e-04 - root_mean_squared_error: 0.0257 - val_loss: 6.6344e-05 - val_root_mean_squared_error: 0.0081
Epoch 94/100
1806/1806 [==============================] - 10s 5ms/step - loss: 6.6059e-04 - root_mean_squared_error: 0.0257 - val_loss: 6.6420e-05 - val_root_mean_squared_error: 0.0081
Epoch 95/100
1806/1806 [==============================] - 10s 5ms/step - loss: 6.6047e-0

Epoch 34/100
1806/1806 [==============================] - 105s 58ms/step - loss: 7.5798e-04 - root_mean_squared_error: 0.0275 - val_loss: 1.1723e-04 - val_root_mean_squared_error: 0.0108
Epoch 35/100
1806/1806 [==============================] - 106s 59ms/step - loss: 7.7303e-04 - root_mean_squared_error: 0.0278 - val_loss: 5.8336e-05 - val_root_mean_squared_error: 0.0076
Epoch 36/100
1806/1806 [==============================] - 106s 59ms/step - loss: 7.5490e-04 - root_mean_squared_error: 0.0275 - val_loss: 1.2037e-04 - val_root_mean_squared_error: 0.0110
Epoch 37/100
1806/1806 [==============================] - 106s 58ms/step - loss: 7.3927e-04 - root_mean_squared_error: 0.0272 - val_loss: 5.4914e-05 - val_root_mean_squared_error: 0.0074
Epoch 38/100
1806/1806 [==============================] - 105s 58ms/step - loss: 7.5720e-04 - root_mean_squared_error: 0.0275 - val_loss: 2.0399e-04 - val_root_mean_squared_error: 0.0143
Epoch 39/100
1806/1806 [==============================] - 106s 59

Epoch 78/100
1806/1806 [==============================] - 107s 59ms/step - loss: 6.8506e-04 - root_mean_squared_error: 0.0262 - val_loss: 1.5662e-04 - val_root_mean_squared_error: 0.0125
Epoch 79/100
1806/1806 [==============================] - 108s 60ms/step - loss: 6.7965e-04 - root_mean_squared_error: 0.0261 - val_loss: 6.2839e-05 - val_root_mean_squared_error: 0.0079
Epoch 80/100
1806/1806 [==============================] - 107s 59ms/step - loss: 6.6823e-04 - root_mean_squared_error: 0.0259 - val_loss: 1.2344e-04 - val_root_mean_squared_error: 0.0111
Epoch 81/100
1806/1806 [==============================] - 107s 60ms/step - loss: 6.7921e-04 - root_mean_squared_error: 0.0261 - val_loss: 1.1343e-04 - val_root_mean_squared_error: 0.0107
Epoch 82/100
1806/1806 [==============================] - 108s 60ms/step - loss: 6.7667e-04 - root_mean_squared_error: 0.0260 - val_loss: 6.7139e-05 - val_root_mean_squared_error: 0.0082
Epoch 83/100
1806/1806 [==============================] - 107s 59

...

Epoch 52/100
1806/1806 [==============================] - 134s 74ms/step - loss: 6.5825e-04 - root_mean_squared_error: 0.0257 - val_loss: 7.0845e-05 - val_root_mean_squared_error: 0.0084
Epoch 53/100
1806/1806 [==============================] - 134s 74ms/step - loss: 6.5649e-04 - root_mean_squared_error: 0.0256 - val_loss: 6.6607e-05 - val_root_mean_squared_error: 0.0082
Epoch 54/100
1806/1806 [==============================] - 135s 75ms/step - loss: 6.5754e-04 - root_mean_squared_error: 0.0256 - val_loss: 6.1967e-05 - val_root_mean_squared_error: 0.0079
Epoch 55/100
1806/1806 [==============================] - 134s 74ms/step - loss: 6.5528e-04 - root_mean_squared_error: 0.0256 - val_loss: 5.9820e-05 - val_root_mean_squared_error: 0.0077
Epoch 56/100
1806/1806 [==============================] - 134s 74ms/step - loss: 6.5527e-04 - root_mean_squared_error: 0.0256 - val_loss: 7.3192e-05 - val_root_mean_squared_error: 0.0086
Epoch 57/100
1806/1806 [==============================] - 134s 74

Epoch 96/100
1806/1806 [==============================] - 133s 73ms/step - loss: 6.3838e-04 - root_mean_squared_error: 0.0253 - val_loss: 5.5046e-05 - val_root_mean_squared_error: 0.0074
Epoch 97/100
1806/1806 [==============================] - 134s 74ms/step - loss: 6.3730e-04 - root_mean_squared_error: 0.0252 - val_loss: 5.1404e-05 - val_root_mean_squared_error: 0.0072
Epoch 98/100
1806/1806 [==============================] - 133s 74ms/step - loss: 6.3668e-04 - root_mean_squared_error: 0.0252 - val_loss: 5.3367e-05 - val_root_mean_squared_error: 0.0073
Epoch 99/100
1806/1806 [==============================] - 133s 74ms/step - loss: 6.3396e-04 - root_mean_squared_error: 0.0252 - val_loss: 5.0417e-05 - val_root_mean_squared_error: 0.0071
Epoch 100/100
1506/1506 [==============================] - 101s 67ms/step
Model::::::::::::::::::::::::::::::: 0
No. of repeatitions completed: 0
No. of repeatitions completed: 1
No. of repeatitions completed: 2
No. of repeatitions completed: 3
No. of 

No. of repeatitions completed: 1
No. of repeatitions completed: 2
No. of repeatitions completed: 3
No. of repeatitions completed: 4
No. of repeatitions completed: 5
No. of repeatitions completed: 6
No. of repeatitions completed: 7
No. of repeatitions completed: 8
No. of repeatitions completed: 9
No. of repeatitions completed: 10
No. of repeatitions completed: 11
No. of repeatitions completed: 12
No. of repeatitions completed: 13
No. of repeatitions completed: 14
No. of repeatitions completed: 15
No. of repeatitions completed: 16
No. of repeatitions completed: 17
No. of repeatitions completed: 18
No. of repeatitions completed: 19
Model::::::::::::::::::::::::::::::: 11
No. of repeatitions completed: 0
No. of repeatitions completed: 1
No. of repeatitions completed: 2
No. of repeatitions completed: 3
No. of repeatitions completed: 4
No. of repeatitions completed: 5
No. of repeatitions completed: 6
No. of repeatitions completed: 7
No. of repeatitions completed: 8
No. of repeatitions comple

No. of repeatitions completed: 9
[04:53:51] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


No. of repeatitions completed: 10
[04:54:14] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


No. of repeatitions completed: 11
[04:54:37] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then

No. of repeatitions completed: 10
[05:01:48] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


No. of repeatitions completed: 11
[05:02:10] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


No. of repeatitions completed: 12
[05:02:32] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  the

In [25]:
X=np.hstack((CNNFeatures,LSTMFeatures,BiLSTMFeatures,ConvLSTMFeatures))
#X = SelectKBest(score_func=f_regression, k=24).fit_transform(X, y)
ICA = FastICA(n_components=24)
X=ICA.fit_transform(X)
MeanAccuracyT=pd.DataFrame()
StdAccuracyT=pd.DataFrame()
for j in range(0,21,1):
    print('Model:::::::::::::::::::::::::::::::',j)
    if j==0:         
        model=LinearRegression()
        name='1.LinearRegression'
    elif j==1:
        model=Lasso()
        name='2.Lasso'
    elif j==2:
        model=Ridge()
        name='3.Ridge'
    elif j==3:
        model=ElasticNet()
        name='4.ElasticNet'
    elif j==4:
        model=HuberRegressor()
        name='5.HuberRegressor'
    elif j==5:
        model=SGDRegressor()
        name='6.SGDRegressor'
    elif j==6:
        model=TweedieRegressor()
        name='7.TweedieRegressor'
    elif j==7:
        model=AdaBoostRegressor()
        name='8.AdaBoostRegressor'
    elif j==8:
        model=RandomForestRegressor()
        name='9.RandomForestRegressor'
    elif j==9:
        model=GradientBoostingRegressor()
        name='10.GradientBoostingRegressor'
    elif j==10:
        model=LinearSVR()
        name='11.LinearSVR'
    elif j==11:
        model=MLPRegressor()
        name='12.MLP'
    elif j==12:
        model=SVR()
        name='13.SVR'
    elif j==13:
        model=ExtraTreesRegressor()
        name='14.ExtraTreesRegressor'
    elif j==14:
        model=BaggingRegressor()
        name='15.BaggingRegressor'
    elif j==15:
        model=DecisionTreeRegressor()
        name='16.DecisionTreeRegressor'
    elif j==16:
        model=KNeighborsRegressor()
        name='17.KNeighborsRegressor'
    elif j==17:
        model=xgb.XGBRegressor(silent=True)
        name='18.XGB'
    elif j==18:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=xgb.XGBRegressor(silent=True))
        name='19.Lasso_Tweedi_xgb'
    elif j==19:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=LinearSVR())
        name='20.Lasso_Tweedi_LinearSVR'
    elif j==20:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=Lasso())
        name='21.Lasso_Tweedi_Lasso'
    elif j==21:
        name='22.LSTM'
    elif j==22:
        name='23.CNN'
    elif j==23:
        name='24.DMLP' 
    elif j==24:
        name='25.GRU'
    else:
        print('Completed.....................')

    file1='./'+str(name)+"_Accuracy.xlsx"
    file2='./'+str(name)+"_Forecasts.xlsx"
    Forecasts=pd.DataFrame()
    Accuracy=pd.DataFrame()
    for i in range(NumberOfRepeatitions):
        print('No. of repeatitions completed:',i)
        if j<=20:
            ynorm1=Find_Fitness(X,y,lenValidation,lenTest,model)
        elif j==21:
            ynorm1=Find_Fitness_LSTM(X,y,lenValidation,lenTest)
        elif j==22:
            ynorm1=Find_Fitness_CNN(X,y,lenValidation,lenTest)
        elif j==23:
            ynorm1=Find_Fitness_DMLP(X,y,lenValidation,lenTest)
        else:
            ynorm1=Find_Fitness_GRU(X,y,lenValidation,lenTest)
        ynorm=pd.DataFrame(normalizedData.iloc[0:LagLength,0])
        ynorm=ynorm.append(ynorm1,ignore_index = True)
        yhat2=minmaxDeNorm(Residual_Data, ynorm, lenTrain+lenValidation)
        yhat=pd.DataFrame(np.zeros((Timeseries_Data.shape[0],1)))
        for ii in range(lt):
            yhat.iloc[ii,0]=yhat2.iloc[ii,0]+linear_Data.iloc[ii,0]
        Accuracy.loc[i,0],Accuracy.loc[i,1]=findRMSE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,2],Accuracy.loc[i,3]=findSMAPE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,4],Accuracy.loc[i,5]=findMAE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,6],Accuracy.loc[i,7]=findMASE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Forecasts=Forecasts.append(yhat.T,ignore_index = True)
    Accuracy.to_excel(file1,sheet_name='Accuracy',index=False)
    Forecasts.T.to_excel(file2,sheet_name='Forecasts',index=False)
    MeanAccuracy=pd.DataFrame(np.mean(Accuracy))    
    MeanAccuracyT=MeanAccuracyT.append(MeanAccuracy.T, ignore_index = True)
    StdAccuracy=pd.DataFrame(np.std(Accuracy))    
    StdAccuracyT=StdAccuracyT.append(StdAccuracy.T, ignore_index = True)
    del Accuracy
    del Forecasts
    del MeanAccuracy
    del StdAccuracy
MeanAccuracyT.to_excel('All_Model_Mean_Accuracy.xlsx',sheet_name='All_Model_Accuracy',index=False)
del MeanAccuracyT
StdAccuracyT.to_excel('All_Model_Stdev_Accuracy.xlsx',sheet_name='All_Model_Stdev_Accuracy',index=False)
del StdAccuracyT

Model::::::::::::::::::::::::::::::: 0
No. of repeatitions completed: 0
No. of repeatitions completed: 1
No. of repeatitions completed: 2
No. of repeatitions completed: 3
No. of repeatitions completed: 4
No. of repeatitions completed: 5
No. of repeatitions completed: 6
No. of repeatitions completed: 7
No. of repeatitions completed: 8
No. of repeatitions completed: 9
No. of repeatitions completed: 10
No. of repeatitions completed: 11
No. of repeatitions completed: 12
No. of repeatitions completed: 13
No. of repeatitions completed: 14
No. of repeatitions completed: 15
No. of repeatitions completed: 16
No. of repeatitions completed: 17
No. of repeatitions completed: 18
No. of repeatitions completed: 19
Model::::::::::::::::::::::::::::::: 1
No. of repeatitions completed: 0
No. of repeatitions completed: 1
No. of repeatitions completed: 2
No. of repeatitions completed: 3
No. of repeatitions completed: 4
No. of repeatitions completed: 5
No. of repeatitions completed: 6
No. of repeatitions c

No. of repeatitions completed: 11
No. of repeatitions completed: 12
No. of repeatitions completed: 13
No. of repeatitions completed: 14
No. of repeatitions completed: 15
No. of repeatitions completed: 16
No. of repeatitions completed: 17
No. of repeatitions completed: 18
No. of repeatitions completed: 19
Model::::::::::::::::::::::::::::::: 12
No. of repeatitions completed: 0
No. of repeatitions completed: 1
No. of repeatitions completed: 2
No. of repeatitions completed: 3
No. of repeatitions completed: 4
No. of repeatitions completed: 5
No. of repeatitions completed: 6
No. of repeatitions completed: 7
No. of repeatitions completed: 8
No. of repeatitions completed: 9
No. of repeatitions completed: 10
No. of repeatitions completed: 11
No. of repeatitions completed: 12
No. of repeatitions completed: 13
No. of repeatitions completed: 14
No. of repeatitions completed: 15
No. of repeatitions completed: 16
No. of repeatitions completed: 17
No. of repeatitions completed: 18
No. of repeatition

No. of repeatitions completed: 12
[13:01:14] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


No. of repeatitions completed: 13
[13:01:37] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


No. of repeatitions completed: 14
[13:02:01] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  the

No. of repeatitions completed: 13
[13:09:13] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


No. of repeatitions completed: 14
[13:09:35] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


No. of repeatitions completed: 15
[13:09:56] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  the